In [ ]:
# pip install groq pandas markdown

In [3]:
import os
import pandas as pd
from groq import Groq

import markdown

# Initialize the Groq client
client = Groq(
    api_key="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"
)

# Function to call Groq API with LLAMA 3 model
def call_groq_api(prompt, model="llama3-70b-8192"):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model=model,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error calling Groq API: {e}")
        return None

# Function to generate user prompts based on incorrect answers
def generate_prompts_from_csv(df):
    user_prompts = {}

    # Load the CSV files
    quiz_df = pd.read_csv('Module1_Notes_to_Quiz/quiz.csv')
    response_df = pd.read_csv('Module2_Quiz_Analysis/Test_Responce.csv')

    # Iterate through each student's responses
    for index, row in response_df.iterrows():
        enrollment_no = row['Enrollment No']
        incorrect_questions = []

        # Compare each response with the correct answer
        for i, question in enumerate(quiz_df['Question']):
            correct_answer = quiz_df['Correct Answer'][i]
            student_response = row[i + 1]  # Responses start from the second column

            if student_response != correct_answer:
                incorrect_questions.append(question)

        # Generate prompt for the current student only
        prompt = (
            """
            You are a skilled educational writer, known for explaining complex topics in a way that's easy to understand.
            Based on the following questions, generate a detailed, insightful article that balances depth with clarity.
            ## Article Requirements
            - Minimum of 1000 words, focused on providing clear, well-researched, and accurate information.
            - Maintain a conversational yet professional tone that keeps the reader engaged.
            - Explain each topic thoroughly, using simple language while avoiding jargon.
            - Organize the article with subheadings to improve readability and flow.
            - Structure the content with an introduction, body sections, and a conclusion.
            - Use real-world examples or case studies to make concepts relatable and practical.
            - Ensure the article is both educational and enjoyable to read, helping the reader grasp even complex ideas effortlessly.
            - Generate the response in HTML format, ensuring proper use of tags like <h1>, <h2>, <p>, <ul>, <li>, and <strong> where appropriate.

            ## Article Format:
            Generated Title Based on Questions

            Introduction
            - Begin with a captivating introduction that highlights the importance of the topic in relation to the questions.
            - Provide a clear overview of what the reader will learn and why it matters."
            Section 1: Core Concepts Explained
            - Introduce the key concepts related to the questions. Provide simple, clear definitions and offer the necessary background to set the stage for deeper exploration.
            Section 2: In-Depth Answers to Each Question
            - Answer each question comprehensively, breaking down the topic into easy-to-understand parts. Use lists, bullet points, or examples where appropriate to clarify complex points. Bullet points, or examples where appropriate to clarify complex points.
            """
        )

        # Add only incorrect answers for the current student to the prompt
        for question in incorrect_questions:
            prompt += f"{question}"

        prompt += ("""
            Section 3: Real-World Applications and Examples
            - Demonstrate how these concepts apply in the real world. Use relevant case studies, personal stories, or examples to highlight the importance of the topics in practice.

            Section 4: Addressing Common Challenges
            - Discuss any challenges, misconceptions, or mistakes that readers might encounter with these topics. Provide practical solutions or strategies to help them avoid or overcome these issues.

            Conclusion
            Summarize the key takeaways from the article, reinforcing the main insights. End with a thoughtful conclusion that encourages readers to explore the topic further and apply what they've learned.

            """
        )

        user_prompts[enrollment_no] = prompt

    return user_prompts

# Function to create an article for each user based on incorrect answers
def create_articles(user_prompts, output_folder="Module3_Custom_Notes/generated_articles"):
    os.makedirs(output_folder, exist_ok=True)

    for reg_no, prompt in user_prompts.items():
        print(f"Generating article for user {reg_no}...")

        generated_text = call_groq_api(prompt)

        generated_text = markdown.markdown(generated_text)

        html_prefix = """
        <!DOCTYPE html>
        <html>
            <body>
        """
        html_suffix = """
            </body>
        </html>
        """
        generated_text = html_prefix + generated_text + html_suffix

        if generated_text:
            file_name = os.path.join(output_folder, f"user_{reg_no}_article.html")
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(generated_text)
            print(f"Article saved for user {reg_no} as {file_name}\n")
        else:
            print(f"Failed to generate article for user {reg_no}\n")

# Main function to run the script
def main(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path)
        user_prompts = generate_prompts_from_csv(df)
        create_articles(user_prompts)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    csv_file_path = "Module2_Quiz_Analysis/Test_Responce.csv" 
    main(csv_file_path)

/var/folders/9g/qkz1nc2d669_4hr4vncryzfc0000gn/T/ipykernel_12534/2630696639.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  student_response = row[i + 1]  # Responses start from the second column


Generating article for user 23071586...
Article saved for user 23071586 as Module3_Custom_Notes/generated_articles/user_23071586_article.html

Generating article for user 23070530...
Article saved for user 23070530 as Module3_Custom_Notes/generated_articles/user_23070530_article.html

Generating article for user 23070719...
Article saved for user 23070719 as Module3_Custom_Notes/generated_articles/user_23070719_article.html

Generating article for user 23070739...
Article saved for user 23070739 as Module3_Custom_Notes/generated_articles/user_23070739_article.html

Generating article for user 23070769...
Article saved for user 23070769 as Module3_Custom_Notes/generated_articles/user_23070769_article.html

Generating article for user 23070620...
Article saved for user 23070620 as Module3_Custom_Notes/generated_articles/user_23070620_article.html

Generating article for user 23070040...
Article saved for user 23070040 as Module3_Custom_Notes/generated_articles/user_23070040_article.html
